<a href="https://colab.research.google.com/github/esintao/Konrad-og-Esin/blob/main/Chain_of_thought.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from datasets import load_dataset
from transformers import pipeline

# Load model

In [67]:
prompt = "Write an email about an alpaca that likes flan"

model = pipeline(task="text2text-generation", model="google/flan-t5-base")

model(prompt, max_length=128, do_sample=True)

Device set to use cpu
Both `max_new_tokens` (=256) and `max_length`(=128) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'generated_text': "Alpaca: I have an alpaca that likes flan. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he's a very small one. He's a very small one and he'"}]

# Load dataset

In [5]:
import pandas as pd

In [55]:
dataset = load_dataset("derek-thomas/ScienceQA")

df = pd.DataFrame.from_dict(dataset['train'])

df = df[(df['choices'].str.len() == 4) & (df['image'].isna())]

df = df.sample(n=200)



# Prompting without CoT

In [35]:
answers = []
question_list = []

for i, row in df.iterrows():
    question_text = (
        f"Question: {row['question']}\n"
        f"(A) {row['choices'][0]} "
        f"(B) {row['choices'][1]} "
        f"(C) {row['choices'][2]} "
        f"(D) {row['choices'][3]}"
        f"The answer must be formatted as 1 letter, A-D."
    )

    answers.append(model(question_text, max_new_tokens=1, do_sample=False))
    question_list.append(question_text)

answers[0]

[{'generated_text': 'A'}]

In [36]:
question_list[0]

'Question: How long is a rowboat?\n(A) 3 centimeters (B) 3 kilometers (C) 3 meters (D) 3 millimetersThe answer must be formatted as 1 letter, A-D.'

# Prompting with CoT

In [87]:
answers_CoT = []
question_list_CoT = []


# 1. Define a template with a "Few-Shot" example
FEW_SHOT_EXAMPLE = """You are a careful reasoning assistant that answers multiple-choice questions about science.
First, think step by step. Then, give the final answer.

Example Question: What is the main source of energy for Earth?
(A) The Moon (B) The Sun (C) Burning coal (D) Wind
Reasoning: Step 1: Earth's climate and life processes require a massive constant energy source.
Reasoning: Step 2: While wind and coal provide energy, they are secondary sources.
Reasoning: Step 3: The Sun provides the vast majority of energy to Earth via radiation.
Answer: B

---
"""

answers_CoT = []

for i, row in df.iterrows():
    # 2. Construct the prompt with clear newlines and the example
    prompt = (
        f"{FEW_SHOT_EXAMPLE}\n"
        f"Question: {row['question']}\n"
        f"(A) {row['choices'][0]}\n"
        f"(B) {row['choices'][1]}\n"
        f"(C) {row['choices'][2]}\n"
        f"(D) {row['choices'][3]}\n"
        f"Format exactly as:\n"
        f"Reasoning: <step 1>\n"
        f"Reasoning: <step 2>\n"
        f"Reasoning: <step 3>\n"
        f"Answer: <final letter>\n"
        f"Reasoning:" # This "nudge" forces the model to start with the correct word
    )

    # 3. Generate with greedy decoding (do_sample=False) for better formatting consistency
    output = model(prompt, max_new_tokens=256, do_sample=False)

    # We prepend "Reasoning:" back to the text since we used it as a nudge
    generated_text = "Reasoning:" + output[0]['generated_text']
    answers_CoT.append(generated_text)
    question_list_CoT.append(prompt)

answers_CoT[0]

'Reasoning:A cookie is not like the others. The final answer: A ---'

In [86]:
question_list_CoT[0]

'You are a careful reasoning assistant that answers multiple-choice questions about science. First, think step by step. Then, give the final answer. You must format exactly as:\nReasoning: <step 1>\nReasoning: <step 2>\nReasoning: <step 3>\nAnswer: <final letter>\n\nQuestion: Which word is not like the others?\n(A) cookie\n(B) ice cream\n(C) cake\n(D) dirt\nNow answer using the format.'